In [1]:
import torch
from torchvision import models, transforms

In [2]:
model = models.resnet50(pretrained=True)

c:\Users\Andy\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Andy\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Note: you may need to restart the kernel to use updated packages.


In [14]:
from PIL import Image


img = Image.open('./targets/img_001.jpg')

In [15]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img = transform(img)
img.shape

torch.Size([3, 224, 224])

In [16]:
batch_img = torch.unsqueeze(img, 0)
batch_img.shape

torch.Size([1, 3, 224, 224])

In [17]:
model.eval()
with torch.no_grad():
    targets = model(batch_img)
is_it = torch.nn.functional.softmax(targets[0], dim=0)
is_it

tensor([2.4830e-03, 2.6604e-04, 3.0629e-05, 2.7270e-05, 7.8573e-05, 1.2216e-04,
        3.3358e-06, 8.5931e-05, 1.2538e-05, 9.9714e-07, 1.5659e-05, 2.1457e-05,
        2.5498e-05, 1.7559e-06, 1.5591e-05, 1.4726e-05, 1.5181e-05, 1.3390e-04,
        1.2088e-05, 4.4910e-05, 3.5022e-05, 8.5771e-05, 1.2370e-05, 8.6282e-06,
        1.4956e-05, 4.2844e-05, 3.4301e-05, 1.2191e-05, 5.9189e-05, 3.8683e-05,
        5.3345e-06, 2.4622e-06, 2.4101e-06, 4.8767e-05, 2.0202e-03, 7.1393e-05,
        4.3111e-04, 1.5294e-05, 1.6270e-04, 9.1905e-06, 1.8581e-05, 5.8189e-05,
        1.1967e-04, 1.5076e-05, 6.2325e-05, 4.2843e-05, 1.1190e-05, 5.3531e-05,
        6.4427e-06, 4.7836e-06, 6.8742e-05, 3.4553e-04, 1.1282e-04, 3.4778e-05,
        3.0730e-05, 5.2000e-06, 3.9066e-05, 5.1234e-06, 2.7469e-05, 2.2619e-05,
        4.5393e-05, 3.0120e-05, 7.4271e-05, 7.6366e-05, 1.0065e-05, 3.9847e-05,
        4.2859e-05, 3.7422e-06, 4.8629e-05, 3.4250e-05, 7.2366e-07, 2.9299e-04,
        2.1031e-06, 1.2879e-05, 2.2002e-

In [10]:
import pandas as pd

img_Classes = pd.read_csv('https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt', header=None)
img_Classes

,0
0,tench
1,goldfish
2,great white shark
3,tiger shark
4,hammerhead
...,...
995,earthstar
996,hen-of-the-woods
997,bolete
998,ear


In [18]:
topk = 5
prob , img_class_number = torch.topk(is_it,topk)
for i in range(topk):
    likelihood = prob[i].item()
    class_name = img_Classes[0][int(img_class_number[i])]
    print('{}% chance image is a {}'.format(int(likelihood*100), class_name))

27% chance image is a Loafer
12% chance image is a cowboy boot
9% chance image is a clog
6% chance image is a bow tie
5% chance image is a running shoe


In [24]:
import numpy as np
import time
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def benchmark(model, device='cuda' , input_shape=(4, 3, 224, 224), dtype='fp32', nwarmup=5, nruns=20):
    input_data = torch.randn(input_shape)
    input_data = input_data.to(device)
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    #torch.cuda.synchronize() #Comment to avoid cuda compile error for torch
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            features = model(input_data)
            #torch.cuda.synchronize() #Comment to avoid cuda compile error for torch
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, ave batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print("Output features size:", features.size())
    print('Average batch time: %.2f ms'%(np.mean(timings)*1000))

In [25]:
benchmark(model, device='cpu')

Warm up ...
Start timing ...
Iteration 10/20, ave batch time 434.40 ms
Iteration 20/20, ave batch time 453.25 ms
Input shape: torch.Size([4, 3, 224, 224])
Output features size: torch.Size([4, 1000])
Average batch time: 453.25 ms
